In [2]:
import cv2 as cv
import numpy as np
import os
from os.path import isfile, join

class Solution():
    def __init__(self, path, title):
        # image
        self.path = path
        # just the first image in the folder
        self.img = cv.imread(path + '0001.jpg', cv.IMREAD_GRAYSCALE)
        cv.imwrite('Results/original.jpg', self.img)

        if self.img is None:
            raise NameError('image not found')

        self.title = title

    def createVideo(self, title, path, fps):
        height, width = self.img.shape
        fourcc = cv.VideoWriter_fourcc(*'mp4v')
        video=cv.VideoWriter('title1.mov', fourcc, fps, (width,height))

        # note this length and string format is just for our video input
        for i in range(501):
            # images are formatted 0001 to 0500
            if len(str(i)) == 1:
                num = '000' + str(i)
            elif len(str(i)) == 2:
                num = '00' + str(i)
            else:
                num = '0' + str(i)
            img = cv.imread(path + num + '.jpg')

            # write image to video
            video.write(img)

        cv.destroyAllWindows()
        video.release()

    def targetTracking(self, mode='SSD'):
        ## create initial image boundary ##
        # var to store points of rectangle
        points = []

        # define click event Function
        def click_event(event, x, y, flags, params):
            # connect to points
            nonlocal points

            # create display image
            display = self.img.copy()

            # check for left click
            if event == cv.EVENT_LBUTTONDOWN:
                # add to points
                points = [(x,y)]

                # display coords
                print('p1:', x, ' ', y)

            # check for left click release
            elif event==cv.EVENT_LBUTTONUP:
                # add to points
                points.append((x,y))

                # display coords
                print('p2:', x, ' ', y)

                # display box
                cv.rectangle(display, points[0], points[1], (255, 0, 0), 1)
                cv.imshow("Testing Image", display)

        # display image
        cv.imshow('Testing Image', self.img)

        # use click_event to produce area
        cv.setMouseCallback('Testing Image', click_event)

        # wait for a key to be pressed to destroy all windows
        cv.waitKey(0)
        cv.destroyAllWindows()

        # crop image
        if len(points) == 2:
            min_y = min(points[0][1], points[1][1])
            max_y = max(points[0][1], points[1][1])
            min_x = min(points[0][0], points[1][0])
            max_x = max(points[0][0], points[1][0])
            T = self.img[min_y:max_y, min_x:max_x]
            cv.imshow("Template image", T)
            cv.imwrite('Results/'+ 'cropped_img.jpg', T)
            cv.waitKey(0)

        ## creating BoundingBox for each new image ##
        height, width = self.img.shape

        # template image
        t_height, t_width = T.shape

        # search window margins (one third total)
        window_margin_height = height // 6
        window_margin_width = width // 6

        # for each of the 500 images
        for s in range(1,501):
            # images are formatted 0001 to 0500
            if len(str(s)) == 1:
                num = '000' + str(s)
            elif len(str(s)) == 2:
                num = '00' + str(s)
            else:
                num = '0' + str(s)
            name = self.path + num + '.jpg'

            # create resultant image and testing grayscaled image
            result = cv.imread(name)
            test = cv.imread(name, cv.IMREAD_GRAYSCALE)

            # create search window margins
            window_points = [(points[0][0]-window_margin_width, points[0][1]-window_margin_height),(points[1][0]+window_margin_width, points[1][1]+window_margin_height)]

            # draw search window boundary
            cv.rectangle(result, window_points[0], window_points[1],(50,10,0), 1)
            cv.putText(result, 'Range', window_points[0], cv.FONT_HERSHEY_PLAIN, 0.4, (50,10,0), 1)

            # max_val if CC or NCC, min_val if SSD
            max_val = float('-inf')
            min_val = float('inf')
            object_points = [[],[]]

            # exhaustive search through all 500 images
            for y in range(max(0, window_points[0][1]), min(height, window_points[1][1]-1)):
                for x in range(max(0, window_points[0][0]), min(width, window_points[1][0]-1)):
                    if y + t_height in range(height) and x + t_width in range(width):
                        I = test[y:y+t_height, x:x+t_width]
                        if mode == 'SSD':
                            val = self.SSD(I, T)
                            if min_val > val:
                                min_val = val
                                object_points = [(x,y),(x+t_width,y+t_height)]
                                new_T = I
                        else:
                            if mode == 'CC':
                                val = self.CC(I, T)
                            else:
                                val = self.NCC(I, T)

                            if max_val < val:
                                max_val = val
                                object_points = [(x,y),(x+t_width,y+t_height)]
                                new_T = I

            # set new template and window points
            T = new_T
            points = object_points

            # draw image boundary
            cv.rectangle(result, points[0], points[1], (0,100,50), 1)
            cv.putText(result, 'girl_face', points [0], cv.FONT_HERSHEY_PLAIN, 0.4, (0,100,50), 1)

            # save image
            cv.imwrite('Results/'+ num + '.jpg', result)
            print(s)
            # cv.imshow('window', result)
            # cv.waitKey(0)


    ## Metrics ## (could be seperate class?)
    # sum of squared diff
    def SSD(self, I, T):
        total = 0

        diff = I - T
        total = np.sum(diff**2)

        return total

    # cross-correlation
    def CC(self, I, T):
        total = 0

        mult = I*T
        total = np.sum(mult)

        return total

    # normalized cross-correlation
    def NCC(self, I, T):
        total = 0

        I_mean = np.mean(I)
        T_mean = np.mean(T)

        I_norm = I - I_mean
        T_norm = T - T_mean

        numer = np.sum(I_norm * T_norm)
        denom = np.sqrt(np.sum(I_norm**2) * np.sum(T_norm**2))
        total = numer/denom
        return total

# Experiment
girl = Solution('image_girl/','girl')
# girl.createVideo('original', girl.path, 15)
# girl.targetTracking(mode='NCC')
# girl.createVideo('tracked_NCC', 'Results/', 10)
girl.targetTracking(mode='SSD')
girl.createVideo('tracked_SSD', 'Results/', 10)
# girl.targetTracking(mode='CC')
# girl.createVideo('tracked_CC', 'Results/', 10)

p1: 52   25
p2: 95   75
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
